In [ ]:
"""
Template.txt
Incident Case:

 {{ incidentInfo['product_type'] }}  has raised some troubles   at  {{ incidentInfo['open_datetime'] }}.
 This involved as detail below.
  {{ incidentInfo['subject'] }}

List Solution as the folloring steps
{% for index,item in solutionDetailInfo.iterrows() %}
 {{ item['engineer_team' ] }} :  {{ item['solution_des'] }}. 
 {% endfor %}
"""


import pandas as pd
from jinja2 import Template

# Read your Jinja template file
with open('1-incident_content_template.txt', 'r') as file:
    template_text = file.read()

# Create a Jinja template object
template = Template(template_text)

# Define Pandas DataFrame and Series
incidentSr = pd.Series (data={'open_datetime': '2024-01-24 10:00', 
                              'product_type': 'BigQuery',
                              'subject':""" 
                               Why Load job often failed in BigQuery with "Error while reading json data form google cloud storeage"
                              """})
detaiList = {
    'engineer_team': ['Cloud Infrasturcture-Team', 'Data Engineeer-Team'],
    'solution_des': [""" 1.Find the Audit Log for the corresponding BigQuery load job using the following filter in Cloud Logging:
protoPayload.resourceName="projects/PROJECT_ID/jobs/JOB_ID"
protoPayload.methodName="google.cloud.bigquery.v2.JobService.InsertJob" """
                     , """
2.Open the ERROR level entry and navigate to the following object to inspect the contents:
protoPayload -> metadata -> job -> jobStatus -> error
""" ]
}
detailDf = pd.DataFrame(detaiList)
detailDf
# # Render the template with Pandas variables
rendered_text = template.render( incidentInfo=incidentSr,solutionDetailInfo=detailDf )

# # # # Print the rendered text
print(rendered_text)


In [ ]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
from datetime import datetime,timezone

import os
import sys 

import pandas as pd
import numpy as np
from datetime import datetime 
from google.cloud import bigquery
from google.oauth2 import service_account

from google.api_core import exceptions

from dotenv import dotenv_values

In [ ]:
incident_id=4369
# 3743 goog sample AIS

In [ ]:
# def get_incident_content(incident_id,model_id):
#     """
#     Take incident id to get data from incident to create incident as content.
#     Parameters:
#     - incidetn_id (int):  id from app_incident table.
#     - model_id (int):  1=text-bison 2=gemini-pro. The different model have different template.
#     Returns:
#     str: text that describe incident detail that is suitable format for Generative AI to generate summarization.
#     """

In [ ]:

config = dotenv_values('.env')

In [ ]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data_pg(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [ ]:
sql_incident=f" SELECT * FROM view_incident_summarization WHERE id = {incident_id} "
dfIncident=list_data_pg(sql_incident,None,get_postgres_conn())

if dfIncident.empty!=True:
    sr=dfIncident.iloc[0,:]
    
else:
    raise Exception(f"Not found incident id {incident_id}")

print(sr)


In [ ]:
sql_detail=f" SELECT * FROM view_incident_detail_summarization WHERE incident_id = {incident_id} "
dt=list_data_pg(sql_detail,None,get_postgres_conn())
if dt.empty == False:
    dt['no.'] = list(range(1, len(dt) + 1))
    print(dt)
else:
    raise Exception(f"Not found detail of incident id {incident_id}")


In [ ]:
incident_promt = f"""
{sr["open_datetime"]}, IT incident was reported regarding {sr["incident_type"]} on {sr["product_type"]} as {sr["severity"]} severity-level . 
The incident was resolved at {sr["open_datetime"]}.
It involved {sr['model']}, product serial number {sr["serial"]}, manufactured by {sr["brand"]}. 
The incident subject was {sr["subject"]}, incident description is indicated as detailed below.
{sr["description"]}
"""

detail_promtxt=""""
The solution to resolve this incident involved task detail items 
as the following(Each item consists of Engineer-Team and Task-Description respectively).

"""
for index, item in dt.iterrows():
  item_des=f"""{item["no."]}. {item['engineer_team']} :  {item['solution']}.\r\n"""
  detail_promtxt=detail_promtxt+item_des



In [ ]:
# prompt_template = """
#     Write a concise summary of the following text.
#     Return your response in bullet points which covers the key points of the text.

#     ```{text}```

#     BULLET POINT SUMMARY:
# """
# Provide a summary for the following article:
# Provide a summary with about two sentences for the following article:
# Provide a summary with about 3-5 paragraph for the following article:
text=f""""
Write a summary with about bullist and 3-5 paragraph for the following IT-Incident-Case:

{incident_promt }
{detail_promtxt}

Summary:

"""
file_path = "incident_conent.txt"
# Open the file in write mode
with open(file_path, 'w') as file:
    # Write the content to the file
    file.write(text)

print(text)

In [ ]:
# return text